<a href="https://colab.research.google.com/github/RocioLiu/Python_coding/blob/master/4_BUILD_THE_NEURAL_NETWORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BUILD THE NEURAL NETWORK**
Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## **Get Device for Training**
We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let’s check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


## **Define the Class**
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every nn.Module subclass implements the operations on input data in the `forward `method.

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model’s `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(f"logits: {logits}\n")
pred_probab = nn.Softmax(dim=1)(logits)
print(f"pred_probab: {pred_probab}\n")
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

logits: tensor([[0.0000, 0.0211, 0.0062, 0.0000, 0.0579, 0.0000, 0.0693, 0.0000, 0.0000,
         0.0243]], device='cuda:0', grad_fn=<ReluBackward0>)

pred_probab: tensor([[0.0982, 0.1003, 0.0988, 0.0982, 0.1040, 0.0982, 0.1052, 0.0982, 0.0982,
         0.1006]], device='cuda:0', grad_fn=<SoftmaxBackward>)

Predicted class: tensor([6], device='cuda:0')


## **Model Layers**
Lets break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.



In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### **nn.Flatten**
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

`CLASS torch.nn.Flatten(start_dim=1, end_dim=-1)`

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
# another example just to explain the using of start_dim and end_dim
# flatten = nn.Flatten(0, 1)
# flat_image = flatten(input_image)
# print(flat_image.size())

torch.Size([84, 28])


### **nn.Linear**
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

`CLASS torch.nn.Linear(in_features, out_features, bias=True)`

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### **nn.ReLU**
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5041, -0.3254, -0.0724,  0.5028, -0.4527,  0.3200,  0.1772,  0.5594,
         -0.1155,  0.2656,  0.3515,  0.1032,  0.3572,  0.7601,  0.1550, -0.1417,
          0.0688, -0.8251,  0.0325,  0.0914],
        [ 0.4507, -0.6610, -0.2169,  0.2678, -0.4002, -0.2804,  0.5710,  0.6042,
         -0.0133, -0.0253,  0.3999,  0.1328,  0.2600,  0.7753,  0.3664,  0.2271,
         -0.0064, -0.8972,  0.2581, -0.1410],
        [ 0.6237,  0.0648, -0.0608,  0.2527, -0.5957, -0.2589,  0.1912,  0.3633,
          0.0505,  0.1532,  0.2992,  0.2092,  0.4410,  0.6045,  0.3254,  0.0054,
          0.0195, -0.5479,  0.0058, -0.1498]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.5041, 0.0000, 0.0000, 0.5028, 0.0000, 0.3200, 0.1772, 0.5594, 0.0000,
         0.2656, 0.3515, 0.1032, 0.3572, 0.7601, 0.1550, 0.0000, 0.0688, 0.0000,
         0.0325, 0.0914],
        [0.4507, 0.0000, 0.0000, 0.2678, 0.0000, 0.0000, 0.5710, 0.6042, 0.0000,
         0.0000, 0.3999, 0.1328, 0.2600, 0.7753, 0.366

### **nn.Sequential**
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits.shape)
print(logits)

torch.Size([3, 10])
tensor([[-0.0314,  0.2125,  0.2399, -0.0242,  0.4461, -0.0511, -0.5582, -0.2790,
          0.3340, -0.3281],
        [ 0.1169,  0.3461,  0.1693, -0.0612,  0.2160, -0.0243, -0.4837, -0.3432,
          0.2677, -0.3513],
        [ 0.0797,  0.1735,  0.2925, -0.2325,  0.2329, -0.0508, -0.4143, -0.2610,
          0.3902, -0.1498]], grad_fn=<AddmmBackward>)


### **nn.Softmax**
The last linear layer of the neural network returns *logits* - raw values in [-infty, infty] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.



In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0931, 0.1188, 0.1221, 0.0938, 0.1501, 0.0913, 0.0550, 0.0727, 0.1341,
         0.0692],
        [0.1100, 0.1384, 0.1159, 0.0921, 0.1215, 0.0955, 0.0603, 0.0694, 0.1279,
         0.0689],
        [0.1043, 0.1145, 0.1290, 0.0763, 0.1215, 0.0915, 0.0636, 0.0742, 0.1422,
         0.0829]], grad_fn=<SoftmaxBackward>)


## **Model Parameters**
Many layers inside a neural network are *parameterized*, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.
    
In this example, we iterate over each parameter, and print its size and a preview of its values.



In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0053,  0.0047, -0.0055,  ...,  0.0180,  0.0172,  0.0230],
        [-0.0343,  0.0148,  0.0180,  ...,  0.0093,  0.0115,  0.0296]],
       device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0133, 0.0339], device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0068, -0.0134,  0.0205,  ...,  0.0405,  0.0113, -0.0161],
        [ 0.0107,  0.0407, -0.0095,  ...,  0.0376,  0.0090,  0.0350]],
       device='cuda:0', gr